In [1]:
import gym
import torch as T
import numpy as np

In [2]:
class ActorCritic(T.nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.transform = T.nn.Linear(8, 128)
        self.act_layer = T.nn.Linear(128, 4) # Action layer
        self.val_layer = T.nn.Linear(128, 1) # Value layer
        self.log_probs = []
        self.state_vals = []
        self.rewards = []

    def forward(self, state):
        state = T.from_numpy(state).float()
        state = T.nn.functional.relu(self.transform(state))
        state_value = self.val_layer(state)

        act_probs = T.nn.functional.softmax(self.act_layer(state))
        act_dist = T.distributions.Categorical(act_probs)
        action = act_dist.sample()

        self.log_probs.append(act_dist.log_prob(action))
        self.state_vals.append(state_value)

        return action.item()

    def computeLoss(self, gamma=0.99):
        rewards = []
        discounted_reward = 0
        for reward in self.rewards[::-1]:
            discounted_reward = reward + gamma * discounted_reward
            rewards.insert(0, discounted_reward)

        rewards = T.tensor(rewards)
        rewards = (rewards - rewards.mean()) / (rewards.std())

        loss = 0
        for log_probability, value, reward in zip(
            self.log_probs, self.state_vals, rewards):
            advantage = reward - value.item()
            act_loss = -log_probability * advantage
            val_loss = T.nn.functional.smooth_l1_loss(value, reward)
            loss += (act_loss + val_loss)

        return loss

    def clear(self):
        del self.log_probs[:]
        del self.state_vals[:]
        del self.rewards[:]

In [3]:
np.random.seed(0)

In [4]:
env = gym.make("LunarLander-v2")

policy = ActorCritic()
optimizer = T.optim.Adam(policy.parameters(), lr=0.02, betas=(0.9, 0.999))

In [5]:
render = True
np.random.seed(0)
running_reward = 0
for i in np.arange(0, 10000):
    state = env.reset()
    for t in range(10000):
        action = policy(state)
        state, reward, done, _ = env.step(action)
        policy.rewards.append(reward)
        running_reward += reward
        if render and i > 1000:
            #env.render()
            pass
        if done:
            break
    print("Episode {}\tReward: {}".format(i, running_reward))

    # Updating the policy
    optimizer.zero_grad()
    loss = policy.computeLoss(0.99)
    loss.backward()
    optimizer.step()
    policy.clear()

    if i % 20 == 0:
        running_reward = running_reward / 20
        running_reward = 0

/Users/saikat/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
/Users/saikat/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Episode 0	Reward: -320.65657506841114
Episode 1	Reward: -425.64874914703705
Episode 2	Reward: -671.2867424162646
Episode 3	Reward: -1032.281198268248
Episode 4	Reward: -1224.3354097571892
Episode 5	Reward: -1543.1792365484055
Episode 6	Reward: -1927.4910808775028
Episode 7	Reward: -2023.4599189797761
Episode 8	Reward: -2361.9002491621986
Episode 9	Reward: -2677.470775357419
Episode 10	Reward: -2932.068423127369
Episode 11	Reward: -3204.4024449864355
Episode 12	Reward: -3449.3136628102934
Episode 13	Reward: -3465.3763860613317
Episode 14	Reward: -3617.162199366013
Episode 15	Reward: -3736.83983321837
Episode 16	Reward: -3883.140249551331
Episode 17	Reward: -4100.137703945375
Episode 18	Reward: -4303.308164747067
Episode 19	Reward: -4569.71587308837
Episode 20	Reward: -4716.304224574078
Episode 21	Reward: -229.39650150526037
Episode 22	Reward: -253.85683612659807
Episode 23	Reward: -366.8881050265607
Episode 24	Reward: -578.1811229544951
Episode 25	Reward: -733.5076209456713
Episode 26	R

Episode 211	Reward: 171.7957818622016
Episode 212	Reward: 238.08770219947772
Episode 213	Reward: -77.3526721494661
Episode 214	Reward: -283.4342936140408
Episode 215	Reward: -295.4607828229397
Episode 216	Reward: -311.4452457115952
Episode 217	Reward: -286.70668214711424
Episode 218	Reward: -343.2182818598881
Episode 219	Reward: -328.3433611261693
Episode 220	Reward: -343.574282960374
Episode 221	Reward: -258.46769536936836
Episode 222	Reward: -276.7483710298315
Episode 223	Reward: -265.4176639716941
Episode 224	Reward: -228.07231775350894
Episode 225	Reward: -259.79556486178575
Episode 226	Reward: -249.4088496015656
Episode 227	Reward: -211.44638863497826
Episode 228	Reward: -257.5226609857976
Episode 229	Reward: -254.34326404862622
Episode 230	Reward: -294.7235941763971
Episode 231	Reward: -473.3909769270307
Episode 232	Reward: -433.8191460262731
Episode 233	Reward: -368.5729980489066
Episode 234	Reward: -331.32532434124954
Episode 235	Reward: -302.75246016748315
Episode 236	Reward: 

Episode 423	Reward: 392.9451047334496
Episode 424	Reward: 385.89633221563247
Episode 425	Reward: 482.8681636084582
Episode 426	Reward: 699.8619915245498
Episode 427	Reward: 680.0074845802052
Episode 428	Reward: 761.9376356510053
Episode 429	Reward: 784.7225974731493
Episode 430	Reward: 824.9835614165975
Episode 431	Reward: 820.9300220282609
Episode 432	Reward: 1046.8606745019229
Episode 433	Reward: 1124.1834562597974
Episode 434	Reward: 1124.8326674520004
Episode 435	Reward: 1240.2476344576232
Episode 436	Reward: 1228.2431443818507
Episode 437	Reward: 1328.7688469169966
Episode 438	Reward: 1337.6633841747325
Episode 439	Reward: 1458.1325509377043
Episode 440	Reward: 1575.8148029267268
Episode 441	Reward: -15.518545980674205
Episode 442	Reward: -37.547985981361435
Episode 443	Reward: 54.24464316780907
Episode 444	Reward: -31.35691811006454
Episode 445	Reward: 82.96833816572739
Episode 446	Reward: 167.36437276265974
Episode 447	Reward: 241.79246955661347
Episode 448	Reward: 400.340141530

Episode 637	Reward: 1748.7946417618716
Episode 638	Reward: 1888.3950986425273
Episode 639	Reward: 2018.1421509003899
Episode 640	Reward: 2111.9793719889985
Episode 641	Reward: 12.311310105715478
Episode 642	Reward: 140.45688691877547
Episode 643	Reward: 286.9279330999853
Episode 644	Reward: 278.4677028005197
Episode 645	Reward: 425.4090605861513
Episode 646	Reward: 456.76908834338474
Episode 647	Reward: 709.4270314257514
Episode 648	Reward: 835.4481681025418
Episode 649	Reward: 956.9425783243009
Episode 650	Reward: 953.0717791055563
Episode 651	Reward: 1059.8327192986899
Episode 652	Reward: 1062.9561410254573
Episode 653	Reward: 1191.8145610566353
Episode 654	Reward: 1420.6240369694847
Episode 655	Reward: 1542.953838820715
Episode 656	Reward: 1551.940528422623
Episode 657	Reward: 1520.0288396242065
Episode 658	Reward: 1728.3287377136014
Episode 659	Reward: 1719.9504790979802
Episode 660	Reward: 1856.6767746258467
Episode 661	Reward: 212.7978891226444
Episode 662	Reward: 433.08625550810

...

Episode 8968	Reward: 640.187848020999
Episode 8969	Reward: 853.1191468640914
Episode 8970	Reward: 772.9893882389257
Episode 8971	Reward: 1011.2862320735946
Episode 8972	Reward: 1027.5213869022298
Episode 8973	Reward: 1236.0087627322864
Episode 8974	Reward: 1456.4147855040296
Episode 8975	Reward: 1689.853259078927
Episode 8976	Reward: 1617.6709366163527
Episode 8977	Reward: 1884.738445266323
Episode 8978	Reward: 2110.938641081778
Episode 8979	Reward: 2177.9170728363624
Episode 8980	Reward: 2455.346319922353
Episode 8981	Reward: 236.9897319745049
Episode 8982	Reward: -6.9173404795406555
Episode 8983	Reward: 238.23367950084628
Episode 8984	Reward: 496.4621448875596
Episode 8985	Reward: 394.1677086140446
Episode 8986	Reward: 645.1728212377651
Episode 8987	Reward: 910.864619545466
Episode 8988	Reward: 1081.2759123357864
Episode 8989	Reward: 1316.165258775607
Episode 8990	Reward: 1358.6255326226847
Episode 8991	Reward: 1605.1424340293472
Episode 8992	Reward: 1860.1119923744616
Episode 8993	R

Episode 9176	Reward: 1637.3205699571376
Episode 9177	Reward: 1877.9662821400138
Episode 9178	Reward: 2110.57762025573
Episode 9179	Reward: 2114.3768286904065
Episode 9180	Reward: 2337.191795267672
Episode 9181	Reward: 9.22254402087016
Episode 9182	Reward: 267.21171436764257
Episode 9183	Reward: 80.82827444193879
Episode 9184	Reward: 341.14232128791775
Episode 9185	Reward: 584.7851117686193
Episode 9186	Reward: 569.274361230576
Episode 9187	Reward: 782.7980129153716
Episode 9188	Reward: 1036.6445314277032
Episode 9189	Reward: 1229.7484691145473
Episode 9190	Reward: 1223.3124444194261
Episode 9191	Reward: 1471.2333105855216
Episode 9192	Reward: 1672.9160673895083
Episode 9193	Reward: 1931.6982587680263
Episode 9194	Reward: 2163.767531880135
Episode 9195	Reward: 2404.3270579801
Episode 9196	Reward: 2652.1168488957014
Episode 9197	Reward: 2944.6147254707557
Episode 9198	Reward: 3122.8952729745115
Episode 9199	Reward: 3367.336528823147
Episode 9200	Reward: 3579.802660300426
Episode 9201	Rew

Episode 9384	Reward: 524.5540507265099
Episode 9385	Reward: 790.9190657577228
Episode 9386	Reward: 769.262172032204
Episode 9387	Reward: 772.7966843292794
Episode 9388	Reward: 1011.2632122376465
Episode 9389	Reward: 1048.1574644666796
Episode 9390	Reward: 1109.9770053726218
Episode 9391	Reward: 1128.264472300024
Episode 9392	Reward: 1354.1320454458012
Episode 9393	Reward: 1338.2883970915902
Episode 9394	Reward: 1338.8237796940628
Episode 9395	Reward: 1356.246212519103
Episode 9396	Reward: 1372.7685900869985
Episode 9397	Reward: 1423.7472777107894
Episode 9398	Reward: 1683.2809589311532
Episode 9399	Reward: 1958.1208254512262
Episode 9400	Reward: 2229.3121951705407
Episode 9401	Reward: -6.687712040632192
Episode 9402	Reward: 5.905634652673228
Episode 9403	Reward: 253.1819902115166
Episode 9404	Reward: 236.4213377898356
Episode 9405	Reward: 270.22160090803436
Episode 9406	Reward: 302.00736346681674
Episode 9407	Reward: 549.7146147502572
Episode 9408	Reward: 563.5561632719714
Episode 9409

Episode 9592	Reward: 2485.8030232588135
Episode 9593	Reward: 2727.13455670153
Episode 9594	Reward: 2927.2722976600758
Episode 9595	Reward: 2901.9752119327127
Episode 9596	Reward: 3141.086339680358
Episode 9597	Reward: 3126.128126482414
Episode 9598	Reward: 3363.247754910303
Episode 9599	Reward: 3401.3268170926635
Episode 9600	Reward: 3647.977466110895
Episode 9601	Reward: 258.95135890035095
Episode 9602	Reward: 268.751951178422
Episode 9603	Reward: 532.3073581397406
Episode 9604	Reward: 547.6693484342268
Episode 9605	Reward: 614.6156795642445
Episode 9606	Reward: 871.1332328542666
Episode 9607	Reward: 1145.4359924563644
Episode 9608	Reward: 1438.944846978371
Episode 9609	Reward: 1713.7075867822634
Episode 9610	Reward: 1704.368146533821
Episode 9611	Reward: 1971.4311695177316
Episode 9612	Reward: 2233.2186507358106
Episode 9613	Reward: 2482.4105065892545
Episode 9614	Reward: 2446.884952029683
Episode 9615	Reward: 2664.808165568907
Episode 9616	Reward: 2928.0864861701575
Episode 9617	Rew

Episode 9800	Reward: 3712.6295298815894
Episode 9801	Reward: 208.8998353398418
Episode 9802	Reward: 497.4150329750087
Episode 9803	Reward: 787.8895222536055
Episode 9804	Reward: 1055.4770113299237
Episode 9805	Reward: 1240.5306501295483
Episode 9806	Reward: 1461.2478368381321
Episode 9807	Reward: 1646.7671842131522
Episode 9808	Reward: 1933.085532471988
Episode 9809	Reward: 2122.1798385344664
Episode 9810	Reward: 1625.5931331916029
Episode 9811	Reward: 1913.631297489083
Episode 9812	Reward: 1880.2306384679923
Episode 9813	Reward: 2105.490136497934
Episode 9814	Reward: 2398.4738605252574
Episode 9815	Reward: 2604.182398982984
Episode 9816	Reward: 2832.6895150602736
Episode 9817	Reward: 3053.543890534699
Episode 9818	Reward: 3051.1787363966937
Episode 9819	Reward: 3229.315930701793
Episode 9820	Reward: 3402.3365097104793
Episode 9821	Reward: 271.6664226246404
Episode 9822	Reward: 459.16426181007256
Episode 9823	Reward: 722.6218800743688
Episode 9824	Reward: 1015.1577824227697
Episode 982